In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import re

In [2]:
url = 'https://www.fifa.com/fifaplus/en/tournaments/mens/worldcup/qatar2022/scores-fixtures?country=AZ&wtw-filter=ALL'

In [3]:
driver = webdriver.Edge(executable_path='C:\\Users\Huseyn\Downloads\edgedriver_win64\msedgedriver.exe')
driver.get(url)
time.sleep(5)
html = driver.page_source
driver.close()
soup = BeautifulSoup(html,'html.parser')

In [4]:
len(soup.find_all('div', {'class':'col-xl-12 col-lg-12 ff-pb-24 ff-text-blue-dark col-md-12 col-sm-12'}))

23

In [5]:
all_days = soup.find_all('div', {'class':'col-xl-12 col-lg-12 ff-pb-24 ff-text-blue-dark col-md-12 col-sm-12'})

In [6]:
big_list = []
for day in all_days:
    date = day.find_all('div', {'class': 'matches-container_title__1uTPf'})[0].text
    matches = day.find_all('div', {'class': 'col-lg-12 col-md-6 col-sm-12'})
    for match in matches:
        team1 = match.find_all('div', {'class':re.compile('wtw-teams-horizontally-component_TeamName*')})[0].text
        team2 = match.find_all('div', {'class':re.compile('wtw-teams-horizontally-component_TeamName*')})[1].text
        
        score_1 = match.find_all('div', {'class': 'wtw-teams-horizontally-component_score1__3HTmk'})[0].text
        score_2 = match.find_all('div', {'class': 'wtw-teams-horizontally-component_score2__20sPm'})[0].text
        try:
            penalty_note = match.find_all('p', {'class': 'ff-m-0'})[0].text.replace('\xa0', '')
        except:
            penalty_note = ''
        big_list.append([date, team1, team2, score_1, score_2, penalty_note])

In [7]:
df = pd.DataFrame(big_list, columns = ('date', 'team1', 'team2', 'score1', 'score2', 'penalty_note'))

In [8]:
df['date'] = pd.to_datetime(df['date'])

In [9]:
df = df[df['date'] >= '2022-12-03'] # filter and keep only knockout stage

In [10]:
df['winner_team'] = df.apply(lambda x: x[1] if x[3] > x[4] else x[2] if x[3] < x[4] else x[1] if x[1] in x[5] else x[2] if x[2] in x[5] else 'ERROR', axis = 1)

In [11]:
df[df['winner_team'] == 'ERROR']

,date,team1,team2,score1,score2,penalty_note,winner_team


In [12]:
df['penalty_score_winner'] = df.penalty_note.apply(lambda x: int(re.findall('\d' ,re.findall('\d -' , x)[0])[0]) if x != '' else 0)

In [13]:
df['penalty_score_loser'] = df.penalty_note.apply(lambda x: int(re.findall('\d' ,re.findall('- \d' , x)[0])[0]) if x != '' else 0)

In [14]:
df

,date,team1,team2,score1,score2,penalty_note,winner_team,penalty_score_winner,penalty_score_loser
48,2022-12-03,Netherlands,United States,3,1,,Netherlands,0,0
49,2022-12-03,Argentina,Australia,2,1,,Argentina,0,0
50,2022-12-04,France,Poland,3,1,,France,0,0
51,2022-12-04,England,Senegal,3,0,,England,0,0
52,2022-12-05,Japan,Croatia,1,1,Croatia wins 3 - 1 on penalties,Croatia,3,1
53,2022-12-05,Brazil,Korea Republic,4,1,,Brazil,0,0
54,2022-12-06,Morocco,Spain,0,0,Morocco wins 3 - 0 on penalties,Morocco,3,0
55,2022-12-06,Portugal,Switzerland,6,1,,Portugal,0,0
56,2022-12-09,Croatia,Brazil,1,1,Croatia wins 4 - 2 on penalties,Croatia,4,2
57,2022-12-09,Netherlands,Argentina,2,2,Argentina wins 4 - 3 on penalties,Argentina,4,3


In [15]:
df['loser_team'] = df.apply(lambda x: x['team1'] if x['team1'] != x['winner_team'] else x['team2'] if x['team2'] != x['winner_team'] else 'ERROR', axis = 1)

In [16]:
df['winner_team_match_score'] = df.apply(lambda x: x.loc['score1'] if x.loc['team1'] == x.loc['winner_team'] else x.loc['score2'], axis = 1)
df['loser_team_match_score'] = df.apply(lambda x: x.loc['score1'] if x.loc['team1'] == x.loc['loser_team'] else x.loc['score2'], axis = 1)

In [17]:
df = df.drop(columns = ['team1', 'team2', 'score1', 'score2', 'penalty_note'])

In [18]:
df = df[['date', 'winner_team','loser_team', 'winner_team_match_score', 'loser_team_match_score', 
              'penalty_score_winner', 'penalty_score_loser']]

In [19]:
df.dtypes

date                       datetime64[ns]
winner_team                        object
loser_team                         object
winner_team_match_score            object
loser_team_match_score             object
penalty_score_winner                int64
penalty_score_loser                 int64
dtype: object

In [20]:
df['winner_team_match_score'] = df['winner_team_match_score'].astype(int)
df['loser_team_match_score'] = df['loser_team_match_score'].astype(int)

In [21]:
df['winner_final_pts'] = df.winner_team_match_score + df.penalty_score_winner
df['loser_final_pts'] = df.loser_team_match_score + df.penalty_score_loser

In [22]:
df = df.drop(columns = ['winner_team_match_score', 'loser_team_match_score', 'penalty_score_winner', 'penalty_score_loser'])

In [23]:
df.reset_index(drop = True, inplace = True)

In [24]:
df

,date,winner_team,loser_team,winner_final_pts,loser_final_pts
0,2022-12-03,Netherlands,United States,3,1
1,2022-12-03,Argentina,Australia,2,1
2,2022-12-04,France,Poland,3,1
3,2022-12-04,England,Senegal,3,0
4,2022-12-05,Croatia,Japan,4,2
5,2022-12-05,Brazil,Korea Republic,4,1
6,2022-12-06,Morocco,Spain,3,0
7,2022-12-06,Portugal,Switzerland,6,1
8,2022-12-09,Croatia,Brazil,5,3
9,2022-12-09,Argentina,Netherlands,6,5


In [25]:
df.to_excel('2022_World_Cup_knockout_stage.xlsx', index = False)